In [2]:
import gurobipy as gb
from gurobipy import GRB

# Create a new model
model = gb.Model("Vaccine_Distribution")

# Create sets of airports and vaccination sites
airports = ["Billy_Bishop", "Pearson"]
sites = list(range(1, 30))  # Sites 1-29

# Parameters
doses = {
    "Billy_Bishop": 100000,
    "Pearson": 250000
}

# Transportation costs per dose
cost = {
    ("Billy_Bishop", j): 0.05 if 1 <= j <= 5 else 
                         0.06 if 6 <= j <= 10 else 
                         0.07 if 11 <= j <= 15 else 
                         0.08 if 16 <= j <= 20 else 
                         0.09 if 21 <= j <= 25 else 0.10
    for j in sites
}

cost.update({
    ("Pearson", j): 0.08 if 1 <= j <= 5 else 
                     0.05 if 6 <= j <= 10 else 
                     0.09 if 11 <= j <= 15 else 
                     0.10 if 16 <= j <= 20 else 
                     0.07 if 21 <= j <= 25 else 0.06
    for j in sites
})

# Hospital clinics are sites 1-7, city-run sites are 8-29
hospital_sites = list(range(1, 8))
city_sites = list(range(8, 30))

# Total vaccination capacity is 50,000 per day, 7 days/week
total_weekly_capacity = 50000 * 7

# Hospital sites can administer 4 times as much as city sites
# If x is the amount per city site, then 22x + 7(4x) = total_weekly_capacity
# 22x + 28x = 350000
# 50x = 350000
# x = 7000
city_site_capacity = 7000
hospital_site_capacity = 4 * city_site_capacity  # = 28000

# Create decision variables
x = model.addVars(airports, sites, name="x")

# Set objective: minimize transportation cost
model.setObjective(
    gb.quicksum(cost[(i, j)] * x[i, j] for i in airports for j in sites),
    GRB.MINIMIZE
)

# Constraint: Supply at each airport
for i in airports:
    model.addConstr(gb.quicksum(x[i, j] for j in sites) == doses[i], f"Supply_{i}")

# Constraint: Demand at each city-run site
for j in city_sites:
    model.addConstr(gb.quicksum(x[i, j] for i in airports) == city_site_capacity, f"Demand_city_{j}")

# Constraint: Demand at each hospital clinic
for j in hospital_sites:
    model.addConstr(gb.quicksum(x[i, j] for i in airports) == hospital_site_capacity, f"Demand_hospital_{j}")

# Constraint 1: Difference between the number of doses sent from either airport to sites 1-5
# must be within 4,800 units of each other
model.addConstr(
    gb.quicksum(x["Billy_Bishop", j] for j in range(1, 6)) - 
    gb.quicksum(x["Pearson", j] for j in range(1, 6)) <= 4800,
    "Diff_constraint_1"
)

model.addConstr(
    gb.quicksum(x["Pearson", j] for j in range(1, 6)) - 
    gb.quicksum(x["Billy_Bishop", j] for j in range(1, 6)) <= 4800,
    "Diff_constraint_2"
)

# Constraint 2: Number of doses from Pearson to sites 21-25 must be <= 8 times doses from Billy Bishop to sites 11-15
model.addConstr(
    gb.quicksum(x["Pearson", j] for j in range(21, 26)) <= 
    8 * gb.quicksum(x["Billy_Bishop", j] for j in range(11, 16)),
    "Ratio_constraint_1"
)

# Constraint 3: Number of doses from Billy Bishop to sites 26-29 must be >= 80% of doses from Pearson to sites 16-20
model.addConstr(
    gb.quicksum(x["Billy_Bishop", j] for j in range(26, 30)) >= 
    0.8 * gb.quicksum(x["Pearson", j] for j in range(16, 21)),
    "Ratio_constraint_2"
)

# Add non-negativity constraints (these are implied by Gurobi)
for i in airports:
    for j in sites:
        model.addConstr(x[i, j] >= 0, f"NonNeg_{i}_{j}")

# Optimize the model
model.optimize()

# Print the solution
if model.status == GRB.OPTIMAL:
    print(f"Optimal transportation cost: ${model.objVal:.2f}")
    
    # Print doses transported to each site
    print("\nDoses transported to each vaccination site:")
    print("Site\tBilly Bishop\tPearson\tTotal")
    print("-" * 40)
    
    for j in sites:
        billy_doses = x["Billy_Bishop", j].x
        pearson_doses = x["Pearson", j].x
        total_doses = billy_doses + pearson_doses
        print(f"{j}\t{billy_doses:.0f}\t\t{pearson_doses:.0f}\t\t{total_doses:.0f}")
    
    # Print summary statistics
    print("\nSummary:")
    print(f"Total doses from Billy Bishop: {sum(x['Billy_Bishop', j].x for j in sites):.0f}")
    print(f"Total doses from Pearson: {sum(x['Pearson', j].x for j in sites):.0f}")
    print(f"Total doses transported: {sum(x[i, j].x for i in airports for j in sites):.0f}")
    
    # Print constraints of interest
    print("\nConstraints Status:")
    constraint1 = model.getConstrByName("Ratio_constraint_1")
    constraint2 = model.getConstrByName("Ratio_constraint_2")
    abs_constraint1 = model.getConstrByName("Diff_constraint_1")
    abs_constraint2 = model.getConstrByName("Diff_constraint_2")
    
    print(f"Ratio constraint 1 value: {constraint1.slack:.2f}")
    print(f"Ratio constraint 2 value: {constraint2.slack:.2f}")
    print(f"Absolute value constraint 1 value: {abs_constraint1.slack:.2f}")
    print(f"Absolute value constraint 2 value: {abs_constraint2.slack:.2f}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 93 rows, 58 columns and 213 nonzeros
Model fingerprint: 0x6c9be25b
Coefficient statistics:
  Matrix range     [8e-01, 8e+00]
  Objective range  [5e-02, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 2e+05]
Presolve removed 90 rows and 52 columns
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2445013e+04   4.910390e+03   0.000000e+00      0s
       2    2.4828000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.482800000e+04
Optimal transportation cost: $24828.00

Doses transported to each vaccination site:
Site	Billy Bishop	Pearson	Total
----------------------------------------
1	0		28000		28000
2	0	